<a href="https://colab.research.google.com/github/LidiiaMelnyk95/phdproject/blob/main/fuzzy_matching_big_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
import re

###Writing an ngram function###

In [71]:
def ngrams(string, n = 3):
  string = string.encode('ascii', errors = 'ignore' ).decode()
  chars_to_remove = ['(', ')', '[', ']', '{', '}', '.', ';', ',']
  rx = '[' + re.escape(' '.join(chars_to_remove)) + ']'
  string = re.sub(rx, '', string)
  string = string.replace ('&', 'and')
  string = string.replace('-', ' ')
  string = string.replace(',', ' ')
  string = re.sub(' +',' ', string).strip() #remove extra white spaces
  string = ' ' + string + ' ' #pad names for ngrams
  ngrams = zip(*[string[i:] for i in range(n)])
  return [' '.join(ngram) for ngram in ngrams]



In [72]:
text = ' ich finde das einfach blöd und uninteressant'

In [74]:
xls = pd.ExcelFile('/content/Non_binary_small.xlsx')

sheet_to_df_map = {}
i = 0
for sheet_name in xls.sheet_names:
    sheet_to_df_map[sheet_name] = xls.parse(sheet_name)
    


In [75]:
for key in sheet_to_df_map.keys():
  for i,row in sheet_to_df_map[key].iterrows():
    for m in str(row['Reply']).split('\n'):
      if m == 'nan':
        pass
      elif m != 'nan':
        sheet_to_df_map[key].loc[i, 'Comment'] = m

In [76]:
df = pd.DataFrame()

In [77]:
for key in sheet_to_df_map.keys():
  df = df.append(sheet_to_df_map[key])

In [78]:
print(df['Comment'].head(10))

0    Hey ihr Lieben! <br>Als Info vorab: Ich habe m...
1                                  Also dey, xier, ...
2    @Maus201 Solch &quot;idiologischen Sprachhardc...
3    @piiinkDeluxe Was konkret hat das mit dem Them...
4    @Kjartan Ragnarson Weil ich seine Existenz nic...
5    @Maus201 Fakt ist: Marcel-Jana möchte sich nic...
6    @Timm Giesbers Natürlich. Darum ging es aber g...
7    Bis zu dem wideo dachte ich es wäre gar nicht ...
8    Biologisch gibt es nur zwei Geschlechter und i...
9    In welcher Situation, kann mensch nicht auf Pr...
Name: Comment, dtype: object


In [79]:
ngrams_list = [ ]
text_list = []

In [80]:
for i, row in df.iterrows():
  for line in str(row['Comment']).split('/n'):
    ngrams_list.append(ngrams(line)) #create n_grams list just in case and to check how the function works

###Applying the function and creating tf idf matrix###

In [107]:
values = [ ] #get raw text from string data in the comments
for i, row in df.iterrows():
  for line in str(row['Comment']).split('/n'):
    values.append(line)
print(values[:10])

['Hey ihr Lieben!\xa0<br>Als Info vorab: Ich habe mich mit Marcel-Jana darauf verständigt, dass wenn es nicht anders möglich ist, ich das Pronomen &quot;er&quot; für den Film nutzen kann - auch weil unsere Sprache noch keine pronomenlose Form entwickelt hat. <br><br>Für mich bleibt bis heute vollkommen unverständlich, wieso es Menschen wie Marcel-Jana so schwer gemacht wird, ihre eigene Identität auch selbst zu bestimmen. Gutachten, Anträge, monatelanges Warten und hohe Kosten - nur damit sie offiziell sein können, wer sie sind. Und dabei stelle ich mir die Frage: Wer sonst soll eigentlich beurteilen können, wie man sich selbst fühlt, wie man sich sieht? \u2028Ich würde mir wünschen, dass die Verfahren einfacher werden - auch damit sie die Würde von trans- und nicht-binären Personen besser achten. \u2028<br>Was denkt ihr darüber?', 'Also dey, xier, ...', '@Maus201 Solch &quot;idiologischen Sprachhardcoretypen&quot; die ggf. daran glauben das Wirklichkeit durch Sprache/Sprechakt konstru

In [108]:
vectorizer = TfidfVectorizer(min_df = 1, analyzer = ngrams)
tf_idf_matrix = vectorizer.fit_transform(values)

###Finding close matches with cosine similarity###

In [109]:
import numpy as np
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn
import sparse_dot_topn.sparse_dot_topn as ct

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for sparse-dot-topn: filename=sparse_dot_topn-0.3.1-cp37-cp37m-linux_x86_64.whl size=1582080 sha256=7143c57fd2190c339c6506b08f7aa3ced1205f24c896186802bffe4b31ba6058
  Stored in directory: /root/.cache/pip/wheels/3b/3e/02/4ee8cb28ed8b608d530bc43402518a895db8ce89aff8ca4e1f
Successfully built sparse-dot-topn


In [113]:
def cossim_top(A,B, ntop, lower_bound = 0):
  #transform A and B to csr matrix
  # in case they are already in format acceptable for csr, there will be no overhead
  A = A.tocsr()
  B = B.tocsr()
  M, _ = A.shape
  _, N = B.shape

  idx_dtype = np.int32
  nnz_max = M * ntop
  indptr = np.zeros(M+1, dtype = idx_dtype) #row indices
  indices = np.zeros(nnz_max, dtype=idx_dtype) #column indices
  data = np.zeros(nnz_max, dtype=A.dtype)

  ct.sparse_dot_topn(M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
  
  return csr_matrix((data,indices,indptr),shape=(M,N))

###Check suggested performance speed between ngram based sparse matrix matching and fuzzy wuzzy library###

In [ ]:
!pip install fuzzywuzzy

In [100]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time

In [104]:
print(values)

["['Hey ihr Lieben!\\xa0<br>Als Info vorab: Ich habe mich mit Marcel-Jana darauf verständigt, dass wenn es nicht anders möglich ist, ich das Pronomen &quot;er&quot; für den Film nutzen kann - auch weil unsere Sprache noch keine pronomenlose Form entwickelt hat. <br><br>Für mich bleibt bis heute vollkommen unverständlich, wieso es Menschen wie Marcel-Jana so schwer gemacht wird, ihre eigene Identität auch selbst zu bestimmen. Gutachten, Anträge, monatelanges Warten und hohe Kosten - nur damit sie offiziell sein können, wer sie sind. Und dabei stelle ich mir die Frage: Wer sonst soll eigentlich beurteilen können, wie man sich selbst fühlt, wie man sich sieht? \\u2028Ich würde mir wünschen, dass die Verfahren einfacher werden - auch damit sie die Würde von trans- und nicht-binären Personen besser achten. \\u2028<br>Was denkt ihr darüber?'\n 'Also dey, xier, ...'\n '@Maus201 Solch &quot;idiologischen Sprachhardcoretypen&quot; die ggf. daran glauben das Wirklichkeit durch Sprache/Sprechakt 

In [110]:
t1 = time.time()
print(process.extractOne('trans', values)) 
t = time.time()-t1
print("SELFTIMED:", t)
print("Estimated hours to complete for full dataset:", (t*len(org_names))/60/60)


('Transformer', 90)
SELFTIMED: 5.074434518814087
Estimated hours to complete for full dataset: 7.166229192680783


In [115]:
t1 = time.time()
matches = cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.85)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 5.7840211391448975
  (0, 0)	1.0000000000000007
  (1, 1)	1.0
  (2, 2)	0.9999999999999991
  (3, 3)	0.9999999999999996
  (4, 4)	0.9999999999999997
  (5, 5)	0.9999999999999999
  (6, 6)	0.9999999999999998
  (7, 7)	0.9999999999999997
  (8, 8)	1.0000000000000002
  (9, 9)	1.0000000000000007
  (10, 10)	0.9999999999999986
  (11, 11)	0.9999999999999994
  (12, 12)	1.0000000000000009
  (13, 13)	1.0000000000000004
  (14, 14)	1.0
  (15, 15)	0.9999999999999999
  (16, 16)	1.0
  (17, 17)	1.0000000000000009
  (18, 18)	1.000000000000001
  (19, 19)	0.9999999999999996
  (20, 20)	0.9999999999999999
  (21, 21)	0.9999999999999993
  (22, 22)	1.0
  (23, 23)	1.0
  (24, 24)	0.9999999999999998
  :	:
  (5175, 5175)	1.0000000000000004
  (5176, 5176)	1.0000000000000002
  (5177, 5177)	0.9999999999999988
  (5178, 5178)	1.0000000000000002
  (5179, 5179)	0.9999999999999997
  (5180, 5180)	0.9999999999999998
  (5181, 5181)	0.9999999999999998
  (5182, 5182)	0.9999999999999996
  (5183, 5183)	0.9999999999999991
  (5